In [ ]:
%pip install -q airavata-python-sdk[notebook]
import airavata_jupyter_magic

%authenticate
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC2:cloud,expanse:shared,anvil:shared
%switch_runtime hpc_cpu

: 

In [ ]:
# In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# === Configuration ===
model_path = "/cybershuttle_data/airavata-courses-deepseek-chat/deepseek-math-7b-instruct"
prompt = "Compute the derivative of sin(x) * x^2."  #Change this prompt
# === Load tokenizer & model (offline) ===
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,    # or torch.float16 if preferred
    device_map="auto",
    local_files_only=True
)
model.config.use_cache = True


In [ ]:
input_text = f"User: {prompt}\nAssistant:"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
gen_config = GenerationConfig(
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

output_ids = model.generate(**inputs, generation_config=gen_config)
# decode only the newly generated tokens
generated = tokenizer.decode(
    output_ids[0][ inputs["input_ids"].shape[-1]: ],
    skip_special_tokens=True
).strip()

print("Prompt:", prompt)
print("Response:", generated)